## This notebook is for organizing and converting runresults data from Pillar Nonprofit Network website

In [4]:
#Uncomment the pip line if pandas not already installed

#!pip install pandas
import pandas as pd
import numpy as np

### 1. Load csv file 

#### Manually type in file_name to be converted and output_file_name to be exported. 
#### If you want the output file to be exported to a directory other than the current one, uncomment and type in the path 

In [20]:
file_name = 'Input the file_name to be converted'
output_file_name = 'Input the output_file_name to be exported'

df = pd.read_csv('runresults_200902_pillar.csv',engine='python')
#df = pd.read_csv(file_name,engine='python')
df.head()
#path='Uncomment and input path to be exported to'

,positions_name,positions_url,positions_org,positions_date_posted,positions_logo,positions_location,positions_description,positions_deadline,positions_skills,positions_contact_name,positions_contact_phone,positions_contact_email,positions_time_commitment,positions_hours
0,Children's Health Foundation Youth Philanthrop...,https://pillarnonprofit.ca/volunteer/childrens...,Children's Health Foundation,20-Aug-20,http://www.pillarnonprofit.ca/civicrm/contact/...,"345 Westminster Ave, London, Ontario","Are you a young person, 14-24 years of age, wi...","Wednesday, September 16, 2020",Have the ability to work well in a team enviro...,Ashley Thorne,+1(548) 388-1416,ypg@childhealth.ca,Long-Term (6+ months),1-2 or 5-10 hours per month


In [ ]:
df.shape

## 2. Select and rename columns for target file

#### ! column names with new format needs to be manually added in order to be identified 

In [21]:
column_names=[]

#job title
if 'position_name' in df:
    df.rename(columns = {'position_name':'job_title'}, inplace=True)
elif 'positions_name' in df:
    df.rename(columns = {'positions_name':'job_title'}, inplace=True)

column_names.append('job_title')

#description
if 'position_description' in df:
    df.rename(columns = {'position_description':'description'}, inplace=True)
elif 'positions_description' in df:
    df.rename(columns = {'positions_description':'description'}, inplace=True)
if 'description' in df:
    column_names.append('description')

#application_process
column_names.append('application_process')

#placement_miscellaneous
column_names.append('placement_miscellaneous')

#position_url
if 'positions_url' in df:
    df.rename(columns = {'positions_url':'position_url'}, inplace=True)
if 'position_url' in df:
    column_names.append('position_url')

#####################################################################################
#position_category
if 'positions_category' in df:
    df.rename(columns = {'positions_category':'position_category'}, inplace=True)
elif 'position_interests' in df:
    df.rename(columns = {'position_interests':'position_category'}, inplace=True)
elif 'position_areas_of_interest' in df:
    df.rename(columns = {'position_areas_of_interest':'position_category'}, inplace=True)
if 'position_category' in df:
    column_names.append('position_category')    
    
#name
if 'position_org' in df:
    df.rename(columns = {'position_org':'name'}, inplace=True)
elif 'positions_org' in df:
    df.rename(columns = {'positions_org':'name'}, inplace=True)
if 'name' in df:
    column_names.append('name')
    
#position_dateposted
if 'positions_date_posted' in df:
    df.rename(columns = {'positions_date_posted':'position_dateposted'}, inplace=True)
elif 'positions_dateposted' in df:
    df.rename(columns = {'positions_dateposted':'position_dateposted'}, inplace=True)
if 'position_dateposted' in df:
    column_names.append('position_dateposted')
    
#position_deadline
if 'positions_deadline' in df:
    df.rename(columns = {'positions_deadline':'position_deadline'}, inplace=True)
if 'position_deadline' in df:
    column_names.append('position_deadline')

#position_skills
if 'positions_skills' in df:
    df.rename(columns = {'positions_skills':'position_skills'}, inplace=True)
    column_names.append('position_skills')
#address
if 'positions_location' in df:
    df.rename(columns = {'positions_location':'address'}, inplace=True)
if 'address' in df:
    column_names.append('address')
    
if 'postal_code' in df:
    column_names.append('postal_code')
if 'created_at' in df:
    column_names.append('created_at')
if 'updated_at' in df:
    column_names.append('updated_at')
if 'expiration_date' in df:
    #default: "2100-07-20"
    column_names.append('expiration_date')
if 'organization_id' in df:
    column_names.append('organization_id')
if 'position_phone' in df:
    column_names.append('position_phone')

column_names

['job_title',
 'description',
 'application_process',
 'placement_miscellaneous',
 'position_url',
 'name',
 'position_dateposted',
 'position_deadline',
 'position_skills',
 'address']

## 3. Create new dataframe with modified column names

#### placement_miscellaneous columns can be further modified if needed


In [ ]:
df['placement_miscellaneous']=df['description']
df['application_process'] = (df['positions_contact_name'].fillna('')+ '  '+
                             df['positions_contact_phone'].fillna('') + '  ' +
                             df['positions_contact_email'].fillna(''))
df_placements = df[column_names]

index=0
df_placements['city'] = np.nan
for address in df['address']:       
    if not address is np.nan: 
        line=address.split(',')
        df_placements.loc[index,'city'] = line[-2]
        index += 1

df_placements['province'] = 'ON'
df_placements['position_postedon']= 'Pillar Nonprofit'        
df_placements.head()

###       Add skills&categories column

In [ ]:
# add skills and categories column
for i in range (12):
    if i<6:
        index = '{} {}'.format('skills', i+1)
        df_placements[index] = np.nan
    else:
        index = '{} {}'.format('categories', i-5)
        df_placements[index] = np.nan

# skills and categories update column name
df_placements.rename(columns = {'skills 1':'skills','skills 2':'skills','skills 3':'skills',
                                'skills 4':'skills','skills 5':'skills','skills 6':'skills',
                                'categories 1':'categories','categories 2':'categories',
                               'categories 3':'categories','categories 4':'categories',
                               'categories 5':'categories','categories 6':'categories'}, inplace=True)
df_placements.head()

## 4. Export dataframe as a csv file

#### Choose to export to current file or to a target file

In [24]:
# Export to current file:
df_placements.to_csv(output_file_name, index=False, escapechar="\r")

# Or to export to a target file, uncomment below:
# import os
# output_file = os.path.join(path, output_file_name)
# df_placements.to_csv(output_file, index=False)

In [19]:
df_placements.head()

,job_title,description,application_process,placement_miscellaneous,position_url,name,position_dateposted,position_deadline,position_skills,address,city
0,Children's Health Foundation Youth Philanthrop...,"Are you a young person, 14-24 years of age, wi...",Ashley Thorne +1(548) 388-1416 ypg@childheal...,"Are you a young person, 14-24 years of age, wi...",https://pillarnonprofit.ca/volunteer/childrens...,Children's Health Foundation,20-Aug-20,"Wednesday, September 16, 2020",Have the ability to work well in a team enviro...,"345 Westminster Ave, London, Ontario",London
